# Experiments

With this notebook, you can easily show and experiments and analyze their evaluations. You can replicate the plots from the thesis and plot different features as well as different experiments.

## Loading the experiment

Before we start, lets define whether we want to save plots in this notebook as svg files.

In [1]:
from typing import Optional
import os

save_to: Optional[str] = None
save_tex = True  # if True, inkscape needs to be executable from the terminal


def save(fig, name):
    if save_to is not None:
        path = os.path.join(save_to, experiment_name)
        if not os.path.exists(path):
            os.makedirs(path)
        fig.write_image(os.path.join(path, name + ".svg"))
        if save_tex:
            import subprocess

            output = subprocess.run(
                [
                    "inkscape -D -z --file="
                    + os.path.join(path, name + ".svg")
                    + " --export-pdf="
                    + os.path.join(path, name + ".pdf")
                    + " --export-latex"
                ],
                check=True,
                capture_output=True,
                shell=True,
            )
            print(output)
            with open(os.path.join(path, name + ".pdf_tex"), "r") as x:
                source = x.read()
                replaced = source.replace("−", "-")
            with open(os.path.join(path, name + ".pdf_tex"), "w") as x:
                x.write(replaced)

    return fig


Next, we enter the name of the dataset we like to analyze. The experiment will be downloaded automatically.

In [2]:
from ml2.ltl.ltl_repair.ltl_repair_sep_hier_transformer_experiment import (
    LTLRepSepHierTransformerExperiment,
)

# Set the name of he experiment here
experiment_name = "exp-repair-gen-96-0"

exp = LTLRepSepHierTransformerExperiment.load(experiment_name)


2022-08-18 12:45:16.942062: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-18 12:45:16.942086: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
INFO:ml2.artifact:Found experiment exp-repair-gen-96-0 locally
INFO:ml2.experiment:Initialized experiment with arguments:
aiger_order: ['inputs', 'latches', 'outputs', 'ands']
aiger_unfold_latches: False
aiger_unfold_negations: False
alpha: 0.5
batch_size: 256
beam_size: 1
cache_dataset: True
checkpoint_monitor: val_accuracy_per_sequence
constant_learning_rate: None
custom_pos_enc: True
d_embed_dec: 256
d_embed_enc: 256
d_ff_dec: 1024
d_ff_enc_g: 1024
d_ff_enc_l1: 1024
d_ff_enc_l2: 1024
dataset_name: scpa-repair-gen-96
drop_batch_remainder: True
dropout_dec: 0.0
dropout_enc: 0.0
dtype_float: <dtype: 'floa

The metadatata of this experiment can be shown as follows. It does not include any results.

In [3]:
exp.metadata


{'aiger_order': ['inputs', 'latches', 'outputs', 'ands'],
 'aiger_unfold_latches': False,
 'aiger_unfold_negations': False,
 'alpha': 0.5,
 'batch_size': 256,
 'beam_size': 1,
 'cache_dataset': True,
 'checkpoint_monitor': 'val_accuracy_per_sequence',
 'constant_learning_rate': None,
 'custom_pos_enc': True,
 'd_embed_dec': 256,
 'd_embed_enc': 256,
 'd_ff_dec': 1024,
 'd_ff_enc_g': 1024,
 'd_ff_enc_l1': 1024,
 'd_ff_enc_l2': 1024,
 'dataset_name': 'scpa-repair-gen-96',
 'drop_batch_remainder': True,
 'dropout_dec': 0.0,
 'dropout_enc': 0.0,
 'dtype_float': tf.float32,
 'dtype_int': tf.int32,
 'encode_realizable': True,
 'ff_activation_dec': 'relu',
 'ff_activation_enc_g': 'relu',
 'ff_activation_enc_l1': 'relu',
 'ff_activation_enc_l2': 'relu',
 'fix_local_embed': False,
 'group': None,
 'initial_steps': 0,
 'inputs': ['i0', 'i1', 'i2', 'i3', 'i4'],
 'max_input_length': 128,
 'max_target_length': 128,
 'name': 'exp-repair-gen-96-0',
 'num_heads_dec': 4,
 'num_heads_enc_g': 4,
 'num_he

## Evaluation

Let's have a look at the evaluation of the experiment. 

### Load evaluation

You can see available results as follows:

In [4]:
import os

list(filter(lambda i: (i.find("pipe") == -1), os.listdir(exp.eval_dir)))


['a0.5-bs1', 'a0.5-bs16']

In [5]:
# Set the parameter of the beam search here: a for alpha and bs for beam size. Usually "a0.5-bs16" and "a0.5-bs1" works.
eval = "a0.5-bs16"


The following splits are available

In [6]:
os.listdir(os.path.join(exp.eval_dir, eval))


['val', 'test']

In [7]:
# Set the split that was evaluated here. Usually "test" and "val" works.
split = "test"


This gives us the a log file and a statistics file.

In [8]:
f = os.listdir(os.path.join(exp.eval_dir, eval, split))
samples = int(f[0][f[0].find("-n") + 2 : f[0].find(".")])
f


['stats-n1024.json', 'log-n1024.csv']

We can load the statistics file as follows:

In [9]:
import json

with open(os.path.join(exp.eval_dir, eval, split, "stats-n" + str(samples) + ".json"), "r") as f:
    s = json.load(f)
s


{'samples': 1024,
 'violated': 9525,
 'satisfied': 6726,
 'beam_search_satisfied': 868,
 'match': 545,
 'steps': 64,
 'decoding_error': 68,
 'invalid': 31,
 'error': 31,
 'timeout': 3,
 'sem_accuracy': 0.84765625,
 'accuracy_encoded': 0.84765625,
 'beams_accuracy': 0.8469891411648569}

Let's load all samples from the experiment evaluation and create a dataframe, tht contains all the interesting information.

In [10]:
import os
import pandas as pd

df = (
    pd.read_csv(
        os.path.join(exp.eval_dir, eval, split, "log-n" + str(samples) + ".csv"),
        converters={
            "hash": lambda x: str(x),
            "beam": lambda x: int(x),
            "status": lambda x: str(x),
            "problem": lambda x: str(x),
            "repair_circuit": lambda x: x.replace("\\n", "\n"),
            "target": lambda x: x.replace("\\n", "\n"),
            "prediction": lambda x: x.replace("\\n", "\n"),
        },
    )
    .set_index(["hash", "repair_circuit", "prediction"])
    .drop("target", axis=1)
)
df_split = (
    exp.dataset[split]
    .data_frame.drop_duplicates(["hash", "repair_circuit"])[
        ["hash", "status", "circuit", "realizable", "repair_circuit"]
    ]
    .rename({"status": "old_status", "circuit": "target"}, axis=1)
    .set_index(["hash", "repair_circuit"])
)
df = df.join(df_split, on=["hash", "repair_circuit"], how="left").reset_index()


INFO:ml2.artifact:Found split_data scpa-repair-gen-96 locally
INFO:ml2.ltl.ltl_repair.ltl_repair_data:Read in metadata
INFO:ml2.ltl.ltl_repair.ltl_repair_data:Load data from file:/home/matthias/ml2-storage/ltl-repair/scpa-repair-gen-96/train.csv
INFO:ml2.ltl.ltl_repair.ltl_repair_data:======================SAMPLES READ: 581778 ======================
INFO:ml2.ltl.ltl_repair.ltl_repair_data:Load data from file:/home/matthias/ml2-storage/ltl-repair/scpa-repair-gen-96/val.csv
INFO:ml2.ltl.ltl_repair.ltl_repair_data:======================SAMPLES READ: 72777 ======================
INFO:ml2.ltl.ltl_repair.ltl_repair_data:Load data from file:/home/matthias/ml2-storage/ltl-repair/scpa-repair-gen-96/test.csv
INFO:ml2.ltl.ltl_repair.ltl_repair_data:======================SAMPLES READ: 72805 ======================
INFO:ml2.experiment:Shuffled dataset


We calculate some Levenshtein distances...

In [11]:
from ml2.ltl.ltl_repair.ltl_repair_data import LTLRepairData

df = LTLRepairData.calculate_levenshtein(
    df=df,
    over=[
        {
            "between_1": "repair_circuit",
            "between_2": "target",
            "res": "lev_rt",
        },
        {
            "between_1": "repair_circuit",
            "between_2": "prediction",
            "res": "lev_rp",
        },
        {
            "between_1": "prediction",
            "between_2": "target",
            "res": "lev_pt",
        },
    ],
    set_match=False,
)


... and calculate the size of circuits and specifciation.

In [12]:
from ml2.ltl.ltl_formula import LTLFormula
from ml2.aiger.aiger import parse


def circuit_size(x, c):
    try:
        return parse(x[c]).size
    except Exception:
        return 0


df["prediction"] = df.apply(
    lambda x: ""
    if (x["status"] not in ["Satisfied", "Match", "Violated", "Invalid"])
    else x["prediction"],
    axis=1,
)
df["status"] = df.apply(
    lambda x: "Error" if (x["status"] not in ["Satisfied", "Match", "Violated"]) else x["status"],
    axis=1,
)
df["status"] = df.apply(
    lambda x: "Violated (Copy)"
    if (x["repair_circuit"] == x["prediction"] and x["status"] == "Violated")
    else x["status"],
    axis=1,
)
df["repair_size"] = df.apply(lambda x: circuit_size(x, "repair_circuit"), axis=1)
df["prediction_size"] = df.apply(lambda x: circuit_size(x, "prediction"), axis=1)
df["target_size"] = df.apply(lambda x: circuit_size(x, "target"), axis=1)
df["problem_size"] = df.apply(lambda x: LTLFormula.from_str(x["problem"]).size(), axis=1)
df["lev_imp"] = df.apply(lambda x: x["lev_pt"] - x["lev_rt"], axis=1)


CRITICAL:ml2.aiger.aiger:Sum of number of inputs, latches and ands is greater than max variable indexerror suppressed
CRITICAL:ml2.aiger.aiger:Sum of number of inputs, latches and ands is greater than max variable indexerror suppressed
CRITICAL:ml2.aiger.aiger:Sum of number of inputs, latches and ands is greater than max variable indexerror suppressed
CRITICAL:ml2.aiger.aiger:Sum of number of inputs, latches and ands is greater than max variable indexerror suppressed
CRITICAL:ml2.aiger.aiger:Sum of number of inputs, latches and ands is greater than max variable indexerror suppressed
CRITICAL:ml2.aiger.aiger:Sum of number of inputs, latches and ands is greater than max variable indexerror suppressed
CRITICAL:ml2.aiger.aiger:Sum of number of inputs, latches and ands is greater than max variable indexerror suppressed
CRITICAL:ml2.aiger.aiger:Sum of number of inputs, latches and ands is greater than max variable indexerror suppressed
CRITICAL:ml2.aiger.aiger:Sum of number of inputs, latche

### Display evaluation data

The dataframe we created, now contains the follwing columns:
 - hash: identifies the specification uniquely
 - repair_circuit: the circuit that was fed into the repair model
 - prediction: the predicted circuit of the repair model
 - beam: For beam sizes greater one, this numbers the different beams of one sample evaluation
 - status: the status of the repair model evaluation
 - problem: the specification
 - old_status: the status describing the relationship between repair_circuit, target and spec
 - target: target circuit. satisfies the spec
 - realizable: Whether the sample is realizable
 - lev_rt: Levenshtein distance between repair_circuit and target
 - lev_rp: Levenshtein distance between repair_circuit and prediction
 - lev_pt: Levenshtein distance between prediction and target
 - repair_size: circuit size of the repair_circuit (ands + latches)
 - prediction_size: circuit size of the prediction (ands + latches)
 - target_size: circuit size of the target (ands + latches)
 - problem_size: size of the AST of the specification
 - lev_imp: Levenshtein improvement through repair model (lev_pt-lev_pt). Positive: deteriorated; Negative: improved

In [13]:
df.head(2)


,hash,repair_circuit,prediction,beam,status,problem,old_status,target,realizable,lev_rt,lev_rp,lev_pt,repair_size,prediction_size,target_size,problem_size,lev_imp
0,b6ea77d4041fa9925ca8338b0ed3c726c50813e22bec84...,aag 9 5 1 5 3\n2\n4\n6\n8\n10\n12 18\n1\n1\n1\...,aag 5 5 0 5 0\n2\n4\n6\n8\n10\n1\n1\n1\n0\n0\n...,0,Violated,(((G ((i1) -> ((X ((((o4) & (! (o3))) & (! (o0...,Violated,aag 10 5 1 5 4\n2\n4\n6\n8\n10\n12 20\n13\n1\n...,0,20,40,50,4,0,5,110,30
1,b6ea77d4041fa9925ca8338b0ed3c726c50813e22bec84...,aag 9 5 1 5 3\n2\n4\n6\n8\n10\n12 18\n1\n1\n1\...,aag 7 5 1 5 1\n2\n4\n6\n8\n10\n12 14\n0\n1\n0\...,1,Violated,(((G ((i1) -> ((X ((((o4) & (! (o3))) & (! (o0...,Violated,aag 10 5 1 5 4\n2\n4\n6\n8\n10\n12 20\n13\n1\n...,0,20,35,43,4,2,5,110,23


Let's first see, how many samples over all its beams are satisfied in this evaluation?
If one beam is satisfied, we count the sample as satisfied.

In [14]:
pd.DataFrame(
    df.groupby(["hash", "repair_circuit"])
    .apply(lambda x: "Match" in x["status"].values)
    .value_counts()
).rename({0: "Match?"}, axis=1).join(
    pd.DataFrame(
        df.groupby(["hash", "repair_circuit"])
        .apply(lambda x: "Satisfied" in x["status"].values or "Match" in x["status"].values)
        .value_counts()
    ).rename({0: "Satisfied?"}, axis=1)
)


,Match?,Satisfied?
True,545,868
False,479,156


Next, we want to know, how many beams of each sample satisfy the specification. We take the mean over all samples.

In [15]:
def h(x):
    sat = 0
    try:
        sat = sat + x["status"].value_counts()["Satisfied"]
    except Exception:
        pass
    try:
        sat = sat + x["status"].value_counts()["Match"]
    except Exception:
        pass
    return sat


counted = df.groupby(["hash", "repair_circuit"]).apply(h).reset_index()
print("correct beams per sample: " + str(counted[0].mean()))
print("correct beams per solved sample: " + str(counted[counted[0] != 0][0].mean()))


correct beams per sample: 6.568359375
correct beams per solved sample: 7.748847926267281


### Aggregating Beams

As mentioned before, because of the beam search we get multiple results per sample. Because all beams are distinct, the chance of all beams satisfying the spec is naturally low. In fact, only one beam can be a "Match". Because of this circumstance, some statistics over the evaluation do not reflect the performance of the model accurately. 

We aggregate all beams from the same sample to one evaluation, to show what the model is actually able to achieve. 

We have different options to aggregate beams. 
 - All columns that do not depend on the prediction should be equal through all beams of the same sample.
 - If `best` is set, we select a randomly chosen beam from each sample, that has the best result. E.g. If no beam of a sample produced a "Match", but several beams produced a "Satisfied", we sample from all Satisfied beams. This reflects the models performance best, but discards all information from other beams.
 - If `best` is not set, we treat the columns differently. For some fields it makes sense to calculate the mean, all others are selected from beams with `best` performance.
 - The parameter `round_mean` is self-explanatory and makes sense for plotting an averaged column as categorical data 

We create the following dataframes once:
 - `df_agg_best = aggregate(df, best=True)`
 - `df_agg_mean = aggregate(df, best=False)`
 - `df_agg_mean_round = aggregate(df, best=False, round_mean=True)`


In [16]:
import pandas as pd
from typing import List


def aggregate(
    df: pd.DataFrame,
    best: bool = True,
    round_mean: bool = False,
    same: List[str] = [
        "hash",
        "repair_circuit",
        "problem",
        "target",
        "realizable",
        "old_status",
        "lev_rt",
        "repair_size",
        "target_size",
        "problem_size",
    ],
    groupby: List[str] = ["hash", "repair_circuit"],
) -> pd.DataFrame:
    def agg_helper(x):
        def all_same(col):
            for _, a in x.iterrows():
                if a[col] != x[col].iloc[0]:
                    raise AssertionError

        def best_status():
            if "Match" in list(x["status"].value_counts().index):  # MATCH
                return x[x["status"] == "Match"].sample()
            elif "Satisfied" in list(x["status"].value_counts().index):  # SATISFIED
                return x[x["status"] == "Satisfied"].sample()
            elif "Violated (Copy)" in list(x["status"].value_counts().index):  # VIOLATED COPY
                return x[x["status"] == "Violated (Copy)"].sample()
            elif "Violated" in list(x["status"].value_counts().index):  # VIOLATED
                return x[x["status"] == "Violated"].sample()
            else:
                return x.sample()

        all(all_same(i) for i in same)
        ret = best_status()
        round_ = round if round_mean else (lambda x: x)
        if not best:
            ret["lev_rp"] = [round_(x["lev_rp"].mean())]
            ret["lev_pt"] = [round_(x["lev_pt"].mean())]
            ret["prediction_size"] = [round_(x["prediction_size"].mean())]
            ret["lev_imp"] = [round_(x["lev_imp"].mean())]
        return ret

    return df.groupby(groupby).apply(agg_helper).reset_index(drop=True)


### Plotting bins

We first provide a function to count the occurrences of each status based on some categorical column. If set, we can also scale each category/bin to sum up to one

In [17]:
df_agg_best = aggregate(df, best=True)
df_agg_mean = aggregate(df, best=False)
df_agg_mean_round = aggregate(df, best=False, round_mean=True)


In [18]:
import pandas as pd


def count(df: pd.DataFrame, by: str, scale: bool, with_copy: bool = True, status: str = "status"):
    dfs = [
        df[df[status] == "Match"].rename({"hash": "Match"}, axis=1).groupby(by).count()["Match"],
        df[df[status] == "Satisfied"]
        .rename({"hash": "Satisfied"}, axis=1)
        .groupby(by)
        .count()["Satisfied"],
        df[df[status] == "Violated"]
        .rename({"hash": "Violated"}, axis=1)
        .groupby(by)
        .count()["Violated"],
    ]
    if with_copy:
        dfs.append(
            df[df[status] == "Violated (Copy)"]
            .rename({"hash": "Violated (Copy)"}, axis=1)
            .groupby(by)
            .count()["Violated (Copy)"]
        )
    dfs.append(
        df[df[status] == "Error"].rename({"hash": "Error"}, axis=1).groupby(by).count()["Error"]
    )
    counted = (
        pd.concat(
            dfs,
            axis=1,
        )
        .fillna(0)
        .reset_index()
    )
    counted["Sum"] = counted.apply(
        lambda x: (
            x["Match"]
            + x["Satisfied"]
            + x["Violated"]
            + (x["Violated (Copy)"] if with_copy else 0)
            + x["Error"]
        ),
        axis=1,
    )
    total_sum = counted["Sum"].max()
    if scale:
        counted["Match"] = counted["Match"].astype(float)
        counted["Satisfied"] = counted["Satisfied"].astype(float)
        counted["Violated"] = counted["Violated"].astype(float)
        if with_copy:
            counted["Violated (Copy)"] = counted["Violated (Copy)"].astype(float)
        counted["Error"] = counted["Error"].astype(float)
        counted["Sum"] = counted["Sum"].astype(float)
        for i, row in counted.iterrows():
            sum = row["Sum"]
            counted.at[i, "Match"] = float(row["Match"]) / sum
            counted.at[i, "Satisfied"] = float(row["Satisfied"]) / sum
            counted.at[i, "Violated"] = float(row["Violated"]) / sum
            if with_copy:
                counted.at[i, "Violated (Copy)"] = float(row["Violated (Copy)"]) / sum
            counted.at[i, "Error"] = float(row["Error"]) / sum
            counted.at[i, "Sum"] = float(row["Sum"]) / total_sum
    return counted


Let's have a look at the aggregated data. We print the number of samples per status considering the best evaluation per beam

In [19]:
merged = count(df_agg_best, by="problem_size", scale=False)
print("Match: " + str(merged["Match"].sum()))
print("Satisfied: " + str(merged["Satisfied"].sum()))
print("Violated: " + str(merged["Violated"].sum()))
print("Violated (Copy): " + str(merged["Violated (Copy)"].sum()))
print("Error: " + str(merged["Error"].sum()))


Match: 545.0
Satisfied: 323.0
Violated: 125.0
Violated (Copy): 31.0
Error: 0.0


Now we define the function to create a bar plot, that shows what percentage/amount of samples has which status in each bin. The column on which the bins are based on can be given as a parameter. 

In [20]:
import plotly.graph_objects as go
import plotly.express as px
import os
from scipy import signal


def plot_accuracy(
    df: pd.DataFrame,
    column: str,
    scale: bool = True,
    amount: bool = True,
    force_smooth: bool = False,
    max: int = None,
    with_copy: bool = True,
    status: str = "status",
):

    merged_ = count(df, by=column, scale=scale, with_copy=with_copy, status=status)
    if max is not None:
        merged_ = merged_[merged_[column] <= max]
        if merged_[column].loc[len(merged_.index) - 1] < max:
            line = [max, 0.0, 0.0, 0.0, 0.0, 0.0]
            if with_copy:
                line.append(0.0)
            merged_.loc[len(merged_.index)] = line
    if merged_[column].max() - merged_[column].min() > 100:
        force_smooth = True
    w_size = int((((merged_[column].max() - merged_[column].min()) / 25) // 2) * 2 + 1)
    win = signal.windows.hann(w_size)
    amount_df = (
        merged_["Sum"]
        if not force_smooth
        else signal.convolve(merged_["Sum"], win, mode="same") / sum(win)
    )
    data = [
        go.Bar(
            name="match",
            x=merged_[column],
            y=merged_["Match"],
            marker_color="green",
        ),
        go.Bar(
            name="satisfied",
            x=merged_[column],
            y=merged_["Satisfied"],
            marker_color="lightgreen",
        ),
        go.Bar(
            name="violated",
            x=merged_[column],
            y=merged_["Violated"],
            marker_color="red",
        ),
    ]

    if with_copy:
        data.append(
            go.Bar(
                name="violated (copy)  ",
                x=merged_[column],
                y=merged_["Violated (Copy)"],
                marker_color="orange",
            )
        )
    data.append(
        go.Bar(
            name="error",
            x=merged_[column],
            y=merged_["Error"],
            marker_color="purple",
        )
    )

    if amount:
        data.append(
            go.Scatter(
                name="No. samples" + (" (smoothed)" if force_smooth else "") + "    ",
                x=merged_[column],
                y=amount_df,
                marker_color="black",
                mode="lines",
                line_shape="spline",
            ),
        )

    fig = go.Figure(data=data)
    fig.update_layout(
        barmode="stack",
        showlegend=True,
        legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5),
        colorscale={"sequential": px.colors.qualitative.G10},
        height=260,
    )
    fig.show()
    fig.update_layout(
        paper_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=0, r=10, t=10, b=0),
    )

    return fig


Let's plot some features. We give some exemplary usage and you can try other parameters

In [21]:
# Levenshtein between repair and target as bins. Aggregated over beams.
save(plot_accuracy(df_agg_best, column="lev_rt", max=100), "bins_lev_rt")


In [22]:
# Levenshtein between repair and target as bins. Not aggregated over beams.
plot_accuracy(df, column="lev_rt")


In [23]:
# prediction size as bins. Aggregated over beams, size is from best beam, not from average
plot_accuracy(df_agg_best, column="prediction_size")


In [24]:
# Levenshtein between prediction and target as bins. Aggregated over beams, distance is averaged and rounded over all beams
plot_accuracy(df_agg_mean_round, column="lev_pt")


In [25]:
# Problem size as bins. Aggregated over beams.
save(plot_accuracy(df_agg_best, column="problem_size", amount=True, max=180), "bins_spec_size")


In [26]:
# Problem size as bins. Aggregated over beams.
save(
    plot_accuracy(df_agg_best, column="target_size", scale=True, amount=True),
    "bins_target_size",
)


In [27]:
# Improvement of Levenshtein distance as bins. Aggregated over beams, improvement taken from the best beam, not averaged.
plot_accuracy(df_agg_best, column="lev_imp")


In [28]:
# Improvement of Levenshtein distance as bins. Aggregated over beams, improvement is averaged over all beams.
plot_accuracy(df_agg_mean_round, column="lev_imp")


### Distribution Plots

With a [Violin Plot](https://en.wikipedia.org/wiki/Violin_plot), we can intuitively show the distribution (probability density) of a column. We show separate the violin plots into different groups, such as the value of `status`. Further, we can split each plot in half, for realizable und unrealizable samples. You can show a [box plot](https://en.wikipedia.org/wiki/Box_plot) inside each violin plot. Note that, without a box plot it is not possible to extract min or max values from the plot as for a violin plot, the distribution is smoothed with a [KDE](https://en.wikipedia.org/wiki/Box_plot).

In [29]:
import plotly.graph_objects as go
import pandas as pd


def violin(
    df: pd.DataFrame, group: str, dist: str, split_realizable: bool = True, box: bool = False
):
    fig = go.Figure()
    if split_realizable:
        fig.add_trace(
            go.Violin(
                y=df[group][df["realizable"] == 0],
                x=df[dist][df["realizable"] == 0],
                legendgroup="unrealizable",
                # scalegroup="unrealizable",
                name="unrealizable",
                side="negative",
            )
        )
        fig.add_trace(
            go.Violin(
                y=df[group][df["realizable"] == 1],
                x=df[dist][df["realizable"] == 1],
                legendgroup="realizable",
                # scalegroup="realizable",
                name="realizable",
                side="positive",
            )
        )
    else:
        fig.add_trace(
            go.Violin(
                y=df[group],
                x=df[dist],
            )
        )
    fig.update_traces(
        meanline_visible=True, points=False, orientation="h", box_visible=box, width=0.8
    )
    fig.update_layout(
        showlegend=split_realizable,
        legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5),
        colorscale={"sequential": px.colors.qualitative.G10},
        height=260,
    )
    fig.show()
    fig.update_layout(
        paper_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=0, r=10, t=10, b=0),
    )
    return fig


We show different configurations and you can try other parameters.

In [30]:
# Show status as group, but only "Match", "Satisfied". Distribution of Levenshtein improvement. Show both realizable and unrealizable samples separately
df_cp = df_agg_best.copy()

df_cp["status"] = df_cp.apply(
    lambda x: "Violated"
    if (x["status"] in ["Violated", "Violated (Copy)", "Error"])
    else x["status"],
    axis=1,
)
df_cp["status"] = df_cp.apply(
    lambda x: "Satisfied" if (x["status"] in ["Satisfied", "Match"]) else x["status"],
    axis=1,
)
df_cp = df_cp[(df_cp["lev_imp"] > -60) & (df_cp["lev_imp"] < 40)]

save(
    violin(
        df_cp,
        group="status",
        dist="lev_imp",
        split_realizable=True,
        box=False,
    ),
    "violin_imp",
)


In [31]:
# Show status as group, but only "Match", "Satisfied", "Violated". Distribution of Levenshtein distance between repair and target. Show box plot too.
save(
    violin(
        df_agg_best[
            (df_agg_best["status"] == "Match")
            | (df_agg_best["status"] == "Satisfied")
            | (df_agg_best["status"] == "Violated")
        ],
        group="status",
        dist="lev_rt",
        split_realizable=False,
        box=True,
    ),
    "violin_lev_rt",
)


In [32]:
# Show status as group, but only "Match", "Satisfied", "Violated". Distribution of problem size. Show box plot too.
fig = violin(
    df_agg_best[
        (df_agg_best["status"] == "Match")
        | (df_agg_best["status"] == "Satisfied")
        | (df_agg_best["status"] == "Violated")
    ],
    group="status",
    dist="problem_size",
    split_realizable=False,
    box=True,
)


## Pipeline Evaluation (Improving Neural Circuit Synthesis)

Now we look at the pipeline evaluations:

Let's first load a model / experiment:

In [33]:
from ml2.ltl.ltl_repair.ltl_repair_sep_hier_transformer_experiment import (
    LTLRepSepHierTransformerExperiment,
)
import pandas as pd

# Set the name of he experiment here
experiment_name = "exp-repair-gen-96-0"

exp = LTLRepSepHierTransformerExperiment.load(experiment_name)


INFO:ml2.artifact:Found experiment exp-repair-gen-96-0 locally
INFO:ml2.experiment:Initialized experiment with arguments:
aiger_order: ['inputs', 'latches', 'outputs', 'ands']
aiger_unfold_latches: False
aiger_unfold_negations: False
alpha: 0.5
batch_size: 256
beam_size: 1
cache_dataset: True
checkpoint_monitor: val_accuracy_per_sequence
constant_learning_rate: None
custom_pos_enc: True
d_embed_dec: 256
d_embed_enc: 256
d_ff_dec: 1024
d_ff_enc_g: 1024
d_ff_enc_l1: 1024
d_ff_enc_l2: 1024
dataset_name: scpa-repair-gen-96
drop_batch_remainder: True
dropout_dec: 0.0
dropout_enc: 0.0
dtype_float: <dtype: 'float32'>
dtype_int: <dtype: 'int32'>
encode_realizable: True
ff_activation_dec: relu
ff_activation_enc_g: relu
ff_activation_enc_l1: relu
ff_activation_enc_l2: relu
fix_local_embed: False
group: None
initial_steps: 0
inputs: ['i0', 'i1', 'i2', 'i3', 'i4']
max_input_length: 128
max_target_length: 128
name: exp-repair-gen-96-0
num_heads_dec: 4
num_heads_enc_g: 4
num_heads_enc_l1: 4
num_head

Here we print all available  evaluated pipeline for this model.

In [34]:
import os

print(
    list(
        filter(
            lambda i: (i.find("pipe") != -1) and i.find(".html") == -1, os.listdir(exp.eval_dir)
        )
    )
)


['pipe-a0.5:0.5-bs16:16-krandom-dval-r5-s2048', 'pipe-a0.5:0.5-bs16:16-kbest-dval-r5-s1024', 'pipe-a0.5:0.5-bs16:1-kall-dval-r10-s1024', 'pipe-a0.5:0.5-bs1:1-kall-dval-r10-s1024', 'pipe-a0.5:0.5-bs16:16-krandom-dtimeouts-r5-s2048', 'pipe-a0.5:0.5-bs16:16-kall-dsyntcomp-r2-s350', 'pipe-a0.5:0.5-bs16:16-kbest-dtest-r5-s1024', 'pipe-a0.5:0.5-bs16:16-kall-djarvis-r2-s200', 'pipe-a0.5:0.5-bs16:16-krandom-dsyntcomp-r5-s350', 'pipe-a0.5:0.5-bs4:4-kbest-dval-r10-s1024', 'pipe-a0.5:0.5-bs4:4-kbest-dtest_fixed-r10-s2048', 'pipe-a0.5:0.5-bs16:16-kbest-dtest_fixed-r5-s2048', 'pipe-a0.5:0.5-bs16:16-krandom-djarvis-r5-s200']


### Results

We define the figure for plotting the improvement of the accuracy after each iteration of the pipeline

In [35]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from typing import List


def fig_pipeline(
    traces: List[str], pipes: List[str], height: int = 360, show_keep: bool = True
) -> go.Figure:
    fig_over = go.Figure()
    for filename in pipes:
        filename = os.path.join(exp.eval_dir, filename)
        stats = pd.read_json(
            os.path.join(os.path.join(filename, "summary-stats.json"))
        ).reset_index()
        stats["index"] = list(
            map(
                lambda i: (str(i) + "<br>base model") if i == 0 else (str(i) + "<br>repair model"),
                list(range(len(stats))),
            )
        )
        for t in traces:
            # fix that we removed sample that have too many inputs/outputs
            if filename.find("syntcomp") != -1 and t.find("encoded") == -1:
                current_total = 210.0
                all_total = 346.0
                stats[t + " accuracy"] = stats.apply(lambda x: (x[t + " accuracy"] * current_total) / all_total, axis = 1)
            elif filename.find("jarvis") != -1 and t.find("encoded") == -1:
                current_total = 70.0
                all_total = 189.0
                stats[t + " accuracy"] = stats.apply(lambda x: (x[t + " accuracy"] * current_total) / all_total, axis = 1)
            fig_over.add_trace(
                go.Scatter(
                    x=stats["index"],
                    y=stats[t + " accuracy"],
                    mode="lines+markers+text",
                    text=stats[t + " accuracy"].round(3),
                    name=t[: t.find(" overall")]
                    + " accuracy"
                    + (
                        (
                            "<br>keep all beams"
                            if filename.find("all") != -1
                            else "<br>keep one beam randomly"
                        )
                        if show_keep
                        else ""
                    ),
                    textposition="bottom center",
                )
            )
    # Edit the layout
    fig_over.update_layout(
        xaxis_title="iterations",
        yaxis_title="accuracy",
        colorscale={"sequential": px.colors.qualitative.G10},
        legend=dict(orientation="h", yanchor="top", y=-0.3, xanchor="center", x=0.5),
        height=height,
    )

    fig_over.update_xaxes(
        ticklabelposition="outside right",
    )

    fig_over.show()
    fig_over.update_layout(
        paper_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=0, r=10, t=10, b=0),
    )
    return fig_over


Here we can plot the Figure. We have to pass the pipeline name and the traces we like to plot.

Traces could be
 - semantic overall: for semantic overall accuracy, showing the accuracy after each iteration of the pipeline.
 - syntactic overall: for syntactic overall accuracy, showing the accuracy after each iteration of the pipeline.
 - semantic : for semantic accuracy, showing the accuracy in each iteration of the pipeline.
 - syntactic : for syntactic accuracy, showing the accuracy in each iteration of the pipeline.
 - encoded overall : for encoded overall accuracy, showing the accuracy after each iteration of the pipeline. Ignores encoding errors (i.e. for benchmarks)
 - encoded : for encoded accuracy, showing the accuracy in each iteration of the pipeline. Ignores encoding errors (i.e. for benchmarks)


In [36]:
f = fig_pipeline(
    traces=["semantic overall"],
    pipes=[
        "pipe-a0.5:0.5-bs16:16-kbest-dtest_fixed-r5-s2048",
    ],
    height=260,
    show_keep=False,
)
# save(f, "pipe_test_fixed_sem")
f
f = fig_pipeline(
    traces=["syntactic overall"],
    pipes=[
        "pipe-a0.5:0.5-bs16:16-kbest-dtest_fixed-r5-s2048",
    ],
    height=260,
    show_keep=False,
)
# save(f, "pipe_test_fixed_syn")
f

In [37]:
f = fig_pipeline(
    traces=["semantic"],
    pipes=[
        "pipe-a0.5:0.5-bs16:16-kbest-dtest_fixed-r5-s2048",
    ],
    height=260,
    show_keep=False,
)
# save(f, "pipe_test_fixed_each_sem")
f
f = fig_pipeline(
    traces=["syntactic"],
    pipes=[
        "pipe-a0.5:0.5-bs16:16-kbest-dtest_fixed-r5-s2048",
    ],
    height=260,
    show_keep=False,
)
# save(f, "pipe_test_fixed_each_syn")
f


In [38]:
f = fig_pipeline(
    traces=["encoded overall"],
    pipes=[
        "pipe-a0.5:0.5-bs16:16-kall-djarvis-r2-s200",
        "pipe-a0.5:0.5-bs16:16-krandom-djarvis-r5-s200",
    ],
    height=260,
)
# save(f, "pipe_jarvis")
f


In [39]:
f = fig_pipeline(
    traces=["semantic overall"],
    pipes=[
        "pipe-a0.5:0.5-bs16:16-krandom-dtimeouts-r5-s2048",
    ],
    height=260,
    show_keep=False,
)
# save(f, "pipe_timeouts")
f


In [40]:
f = fig_pipeline(
    traces=["encoded overall"],
    pipes=[
        "pipe-a0.5:0.5-bs16:16-kall-dsyntcomp-r2-s350",
        "pipe-a0.5:0.5-bs16:16-krandom-dsyntcomp-r5-s350",
    ],
    height=260,
)
# save(f, "pipe_syntcomp")
f


### Evaluated Samples

Here, we have a deeper look at the evaluated samples. We first load a pipeline.

In [41]:
from ml2.ltl.ltl_formula import LTLFormula

pipe = "pipe-a0.5:0.5-bs16:16-kbest-dtest_fixed-r5-s2048"
repeats = int(pipe[pipe.find("-r") + 2 : pipe.find("-s")])
split = str(pipe[pipe.find("-d") + 2 : pipe.find("-r")])
pipe = os.path.join(exp.eval_dir, pipe)
df = pd.read_csv(
    os.path.join(os.path.join(pipe, "summary.csv")),
    na_values="",
    converters={
        "target": lambda x: x.replace("\\n", "\n"), 
    },
)


This function aggregates the pipeline by
1. counting a row as Satisfied if at least one point in the iteration produced the status Satisfied for the row.
2. counting a sample as Satisfied if at least one row (beam) of all rows that belong to the same specification produced the status Satisfied for the row.

In [42]:
from ml2.aiger.aiger import parse


def aggregate_pipe(df: pd.DataFrame, repeats: int, aggregate_to: int) -> pd.DataFrame:
    def circuit_size(x, c):
        try:
            return parse(x[c]).size
        except Exception:
            return 0

    def agg_helper(x):
        def best_status():
            if "Match" in list(x["status"].value_counts().index):  # MATCH
                return x[x["status"] == "Match"].sample()
            elif "Satisfied" in list(x["status"].value_counts().index):  # SATISFIED
                return x[x["status"] == "Satisfied"].sample()
            elif "Violated" in list(x["status"].value_counts().index):  # VIOLATED
                return x[x["status"] == "Violated"].sample()
            else:
                return x.sample()

        ret = best_status()
        return ret

    def h(x):
        stati = []
        for i in range(aggregate_to + 1):
            stati.append(x["status_" + str(i)])
        if "Match" in stati:
            return "Match"
        elif "Satisfied" in stati:
            return "Satisfied"
        elif "Violated" in stati:
            return "Violated"
        else:
            return "Error"

    df["status"] = df.apply(h, axis=1)

    df = (
        df.groupby("hash")
        .apply(agg_helper)
        .reset_index(drop=True)
        .drop(["status_" + str(i) for i in range(repeats + 1)], axis=1)
        .drop(["prediction_" + str(i) for i in range(repeats + 1)], axis=1)
    )
    df["target_size"] = df.apply(lambda x: circuit_size(x, "target"), axis=1)
    df["problem_size"] = df.apply(
        lambda x: 3
        + sum([LTLFormula.from_str(f).size() for f in str(x["guarantees"]).split(",")])
        + sum([LTLFormula.from_str(f).size() for f in str(x["assumptions"]).split(",")]),
        axis=1,
    )
    return df


We produce multiple dataframes, where we aggregate different iterations of the pipeline

In [43]:
df_pipe_agg = []
for i in range(repeats + 1):
    df_pipe_agg.append(aggregate_pipe(df=df, repeats=repeats, aggregate_to=i))


CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Failed to parse aag header: 
CRITICAL:ml2.aiger.aiger:Fai

This defines the per category plot of the status of samples. Contrary to the first chapter of this notebook, we only look at the improvement.

In [44]:
import plotly.graph_objects as go
import plotly.express as px
import os
from scipy import signal


def plot_accuracy_improvement(
    df_larger: pd.DataFrame,
    df_smaller: pd.DataFrame,
    column: str,
    amount: bool = True,
    force_smooth: bool = False,
    max: int = None,
    status: str = "status",
):
    c_agg = count(df_larger, by=column, scale=True, with_copy=False, status=status)
    c_0 = count(df_smaller, by=column, scale=True, with_copy=False, status=status)
    c_diff = c_agg.copy().drop(["Match", "Satisfied"], axis=1)
    c_diff["old_match"] = c_0["Match"]
    c_diff["new_match"] = c_agg["Match"] - c_diff["old_match"]
    c_diff["old_sat"] = (c_0["Match"] + c_0["Satisfied"]) - c_agg["Match"]
    c_diff["old_sat"] = c_diff.apply(lambda x: x["old_sat"] if x["old_sat"] > 0 else 0, axis=1)
    c_diff["h"] = c_agg["Match"]
    c_diff["h"] = c_diff.apply(
        lambda x: (x["old_sat"] + x["new_match"] + x["old_match"]) if x["old_sat"] > 0 else x["h"],
        axis=1,
    )
    c_diff["new_sat"] = (c_agg["Match"] + c_agg["Satisfied"]) - c_diff["h"]
    merged_ = c_diff.drop("h", axis=1)
    if max is not None:
        merged_ = merged_[merged_[column] <= max]
        if merged_[column].loc[len(merged_.index) - 1] < max:
            line = [max, 0.0, 0.0, 0.0, 0.0, 0.0]
            merged_.loc[len(merged_.index)] = line
    if merged_[column].max() - merged_[column].min() > 100:
        force_smooth = True
    w_size = int((((merged_[column].max() - merged_[column].min()) / 25) // 2) * 2 + 1)
    win = signal.windows.hann(w_size)
    amount_df = (
        merged_["Sum"]
        if not force_smooth
        else signal.convolve(merged_["Sum"], win, mode="same") / sum(win)
    )
    data = [
        go.Bar(
            name="Match (new)",
            x=merged_[column],
            y=round(merged_["new_match"], 4),
            marker_color="green",
        ),
        go.Bar(
            name="Satisfied (new)",
            x=merged_[column],
            y=round(merged_["new_sat"], 4),
            marker_color="lightgreen",
        ),
        go.Bar(
            name="Match or Satisfied (unchanged)       x",
            x=merged_[column],
            y=round(merged_["old_match"] + merged_["old_sat"], 4),
            marker_color="rgba(128,128,128,0.7)",  # light grey
        ),
        go.Bar(
            name="Violated or Error (remaining)   ",
            x=merged_[column],
            y=round(merged_["Violated"] + merged_["Error"], 4),
            marker_color="red",
        ),
    ]

    if amount:
        data.append(
            go.Scatter(
                name="No. samples" + (" (smoothed)" if force_smooth else "") + "    ",
                x=merged_[column],
                y=amount_df,
                marker_color="black",
                mode="lines",
                line_shape="spline",
            ),
        )

    fig = go.Figure(data=data)
    fig.update_layout(
        barmode="stack",
        showlegend=True,
        legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5),
        colorscale={"sequential": px.colors.qualitative.G10},
        height=260,
    )
    fig.show()
    fig.update_layout(
        paper_bgcolor="rgba(0,0,0,0)",
        margin=dict(l=0, r=10, t=10, b=0),
    )

    return fig


In [45]:
save(
    plot_accuracy_improvement(
        df_larger=df_pipe_agg[5], df_smaller=df_pipe_agg[0], column="problem_size", amount=True
    ),
    "bins_problem_size_imp_0to5",
)
save(
    plot_accuracy_improvement(
        df_larger=df_pipe_agg[5], df_smaller=df_pipe_agg[0], column="target_size", amount=True
    ),
    "bins_target_size_imp_0to5",
)


In [46]:
save(
    plot_accuracy_improvement(
        df_larger=df_pipe_agg[5], df_smaller=df_pipe_agg[1], column="problem_size", amount=True
    ),
    "bins_problem_size_imp_1to5",
)
save(
    plot_accuracy_improvement(
        df_larger=df_pipe_agg[5], df_smaller=df_pipe_agg[1], column="target_size", amount=True
    ),
    "bins_target_size_imp_1to5",
)


In [47]:
save(
    plot_accuracy_improvement(
        df_larger=df_pipe_agg[1], df_smaller=df_pipe_agg[0], column="problem_size", amount=True
    ),
    "bins_problem_size_imp_0to1",
)
save(
    plot_accuracy_improvement(
        df_larger=df_pipe_agg[1], df_smaller=df_pipe_agg[0], column="target_size", amount=True
    ),
    "bins_target_size_imp_0to1",
)
